In [3]:
import os
import json
import torch
import pandas as pd
from datetime import datetime as dt
import sys
import types
from hmc.utils.dir import create_dir
from hmc.model.arguments import  build

In [4]:
from tqdm.notebook import tqdm

In [5]:
import os
from datetime import datetime

def horario_atual_para_diretorio():
    # Obtém a data e hora atual
    data_hora_atual = datetime.now()
    
    # Formata a data e hora para o formato de diretório desejado
    diretorio = data_hora_atual.strftime('%Y_%m_%d_%H_%M_%S')
    
    return diretorio

# Exemplo de uso
diretorio_formatado = horario_atual_para_diretorio()

In [6]:

base_path = "/mnt/disks/data/fma/trains"
sample_id = "hierarchical_tworoots_dev"


train_path = os.path.join(base_path, sample_id)
torch_path =os.path.join(train_path,'torch')
metadata_path = os.path.join(train_path,"metadata.json")
labels_path = os.path.join(train_path,"labels.json")

model_id = diretorio_formatado

model_path = os.path.join(train_path, model_id)

args = pd.Series({
    "batch_size":32,
    "epochs":10,
    "dropout":0.3,
    'patience':1,
    'shuffle':True,
    'max_queue_size':64,
    "labels_path": labels_path,
    "metadata_path": metadata_path,
    "train_path": os.path.join(torch_path,'train.pth'),
    "test_path": os.path.join(torch_path,'test.pth'),
    "val_path": os.path.join(torch_path,'val.pth'),
    "model_path":model_path
})


In [7]:
create_dir(model_path)

True

In [8]:
from torch.utils.data import Dataset, DataLoader


# Função parse_single_music adaptada
def parse_single_music(data, labels):
    track_id, categories, music = data
    max_depth = len(categories[0])
    data_dict = {}
    for level in range(1, max_depth + 1):
        level_labels = []
        for cat in categories:
            if cat[level-1] != "":
                label = labels[f'label_{level}'][cat[level-1]]
                if label not in level_labels:
                    level_labels.append(label)
            else:
                if len(level_labels) == 0:
                    level_labels.append(-1)
        data_dict[f'label{level}'] = level_labels

    data_dict['features'] = music
    data_dict['track_id'] = track_id

    return data_dict



# Classe personalizada do Dataset
class MusicDataset(Dataset):
    def __init__(self, dataframe, labels):
        self.data = dataframe
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data = self.data.iloc[idx]
        parsed_data = parse_single_music(data, self.labels)
        features = torch.tensor(parsed_data['features'], dtype=torch.float32)
        labels = {key: torch.tensor(value, dtype=torch.long) for key, value in parsed_data.items() if key.startswith('label')}
        track_id = torch.tensor(parsed_data['track_id'], dtype=torch.long)
        return features, labels, track_id

In [9]:
# Create a dummy dataset module with dataset_torch submodule
dataset = types.ModuleType('dataset')
dataset_torch = types.ModuleType('dataset_torch')
dataset.dataset_torch = dataset_torch
sys.modules['dataset'] = dataset
sys.modules['dataset.dataset_torch'] = dataset_torch


dataset_torch.MusicDataset = MusicDataset
sys.modules['dataset_torch'] = dataset_torch

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class OutputNormalization(nn.Module):
    def __init__(self):
        super(OutputNormalization, self).__init__()

    def forward(self, x):
        # Obtemos a classe com a maior probabilidade
        one_hot_encoded = torch.zeros_like(x).scatter_(1, x.argmax(dim=1, keepdim=True), 1.0)
        return one_hot_encoded

class ClassificationLayer(nn.Module):
    def __init__(self, input_shape, size, dropout):
        super(ClassificationLayer, self).__init__()
        self.fc1 = nn.Linear(input_shape, 1024)
        self.dropout1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(1024, 512)
        self.dropout2 = nn.Dropout(dropout)
        self.fc3 = nn.Linear(512, 256)
        self.batch_norm = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(dropout)
        self.fc4 = nn.Linear(256, size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.batch_norm(x)
        x = self.dropout3(x)
        x = self.fc4(x)
        return x

class MusicModel(nn.Module):
    def __init__(self, levels_size, sequence_size=1280, dropout=0.6):
        super(MusicModel, self).__init__()
        self.sequence_size = sequence_size
        self.classification_layers = nn.ModuleList()
        self.output_normalization = OutputNormalization()
        self.batch_norm = nn.BatchNorm1d(sequence_size)
        
        for level, size in levels_size.items():
            if level == 'level1':
                self.classification_layers.append(ClassificationLayer(sequence_size, size, dropout))
            else:
                self.classification_layers.append(ClassificationLayer(sequence_size * 2, size, dropout))

    def forward(self, x):
        outputs = []
        current_output = x
        
        for i, classification_layer in enumerate(self.classification_layers):
            if i > 0:
                normalized_output = self.batch_norm(self.output_normalization(current_output))
                normalized_output = normalized_output.repeat(1, self.sequence_size // normalized_output.shape[1])
                current_input = torch.cat([normalized_output, x], dim=1)
            else:
                current_input = x

            current_output = classification_layer(current_input)
            outputs.append(current_output)
        
        return outputs



In [11]:
with open(args.metadata_path, 'r') as f:
    metadata = json.loads(f.read())
    print(metadata)

{'sequence_size': 1280, 'max_depth': 4, 'levels_size': [2, 30, 16], 'val_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/val', 'train_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/train', 'test_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/test', 'val_torch_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/torch/val.pth', 'train_torch_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/torch/train.pth', 'test_torch_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/torch/test.pth', 'val_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/val.csv', 'train_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/train.csv', 'test_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/test.csv', 'trainset_count': 16791, 'validationset_count': 2007, 'testset_count': 4814}


In [12]:
metadata['trainset_count']

16791

In [13]:
with open(args.labels_path, 'r') as f:
    labels = json.loads(f.read())

In [14]:
levels_size = {'level1': labels['label_1_count'] ,
                   'level2': labels['label_2_count'] ,
                   'level3': labels['label_3_count'] ,
                   'level4': labels['label_4_count'] }

In [15]:
for level, labels in levels_size.items():
    print(level)

level1
level2
level3
level4


In [16]:
depth = len(levels_size)

In [17]:
#ds_train = Dataset(args.trainset_pattern, args.epochs, args.batch_size, levels_size).build(df=False)

In [18]:
#df_train = Dataset(args.trainset_pattern, args.epochs, args.batch_size, levels_size).build(df=True)

In [19]:
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import numpy as np
from hmc.model import MusicModel

import json

def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    return running_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
    
    return running_loss / len(dataloader)


print("========================= Torch =========================")
print("GPUs availables: {}".format(torch.cuda.is_available()))
print("==============================================================")


def run(args: object):
    print(args)

    with open(args.metadata_path, 'r') as f:
        metadata = json.loads(f.read())
        print(metadata)

    with open(args.labels_path, 'r') as f:
        labels = json.loads(f.read())

    levels_size = {'level1': labels['label_1_count'] ,
                   'level2': labels['label_2_count'] ,
                   'level3': labels['label_3_count'] ,
                   'level4': labels['label_4_count'] }

    params: dict = {
        'levels_size': levels_size,
        'sequence_size': metadata['sequence_size'],
        'dropout': args.dropout
    }

    print(params)
    model = MusicModel(levels_size)
    # Exemplo de uso
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    criterion = nn.BCELoss()  # ou outra loss apropriada para seu caso
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Carregar o dataset salvo
    train_dataset = torch.load(args.train_path)
    val_dataset = torch.load(args.val_path)
    
    # Criação do DataLoader
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=args.shuffle)
    val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=args.shuffle)
    
    # Treinamento e avaliação
    num_epochs = 10
    for epoch in range(num_epochs):
        train_loss = train(model, train_dataloader, criterion, optimizer, device)
        val_loss = evaluate(model, val_dataloader, criterion, device)
        print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Validation Loss: {val_loss}')


========================= Torch =========================
GPUs availables: True


In [20]:

time_start = dt.utcnow()
print("[{}] Experiment started at {}".format(id, time_start.strftime("%H:%M:%S")))
print(".......................................")
#print(args)
run(args)
time_end = dt.utcnow()
time_elapsed = time_end - time_start
print(".......................................")
print("[{}] Experiment finished at {} / elapsed time {}s".format(id, time_end.strftime("%H:%M:%S"), time_elapsed.total_seconds()))

[<built-in function id>] Experiment started at 11:06:02
.......................................
batch_size                                                       32
epochs                                                           10
dropout                                                         0.3
patience                                                          1
shuffle                                                        True
max_queue_size                                                   64
labels_path       /mnt/disks/data/fma/trains/hierarchical_tworoo...
metadata_path     /mnt/disks/data/fma/trains/hierarchical_tworoo...
train_path        /mnt/disks/data/fma/trains/hierarchical_tworoo...
test_path         /mnt/disks/data/fma/trains/hierarchical_tworoo...
val_path          /mnt/disks/data/fma/trains/hierarchical_tworoo...
model_path        /mnt/disks/data/fma/trains/hierarchical_tworoo...
dtype: object
{'sequence_size': 1280, 'max_depth': 4, 'levels_size': [2, 30, 16], 'val_p

ValueError: too many values to unpack (expected 2)